In [420]:
import pandas as pd 
import csv
# import seaborn as sns
import matplotlib.pyplot as plt



In [421]:
with open ('file_list.txt', 'r') as f:
    csv_list= f.readlines()
file_dict = {}
for csv in csv_list:
    key='ZippedData/'+csv.strip()
    df_name=csv.strip()
    df_name=df_name.split('.csv.gz')[0]
    df=pd.read_csv(key, compression='gzip')
    file_dict[df_name] = df
file_dict.keys

BadGzipFile: Not a gzipped file (b',i')

### Opening and Combining  Relevant Rotten Tomato Data

In [ ]:
rt_reviews = pd.read_csv('zippedData/rt.reviews.tsv.gz',sep = '\t', compression = 'gzip', encoding = 'latin-1' )

rt_movie_info = pd.read_csv('zippedData/rt.movie_info.tsv.gz',sep = '\t', compression = 'gzip', encoding = 'latin-1' )

rt_reviews.head()

In [ ]:
rt_reviews_info = rt_movie_info[['id', 'director', 'runtime', 'rating', 'theater_date']]

rt_reviews_cleaned = rt_reviews[['id', 'rating', 'critic']]
rt_df = rt_reviews_info.merge(rt_reviews_cleaned, how = 'inner', on = 'id')

rt_df.head(1)

### Normalizing Critic Ratings

In [ ]:
def norm_ratings(rating):
    grades = {"A":10,"B":8,"C":6,"D":4,"F":2,"A+":10,"B+":9,"C+":7,"D+":5,\
              "F+":3,"A-":9,"B-":7,"C-":5,"D-":3,"F-":0, '3 1/2': 3.5, 'T': None, 'N':None, 'R':None}
    
    if rating in grades.keys():
        norm=grades[rating]
        return norm
    elif ('/') in rating:
        r = rating.split('/')
        norm = float(r[0])/float(r[1])*10
        return norm
    elif ('-') in rating and len(rating) > 2:
        r = rating.split('-')
        norm = float(r[0])/float(r[1])*10
        return norm
    else:
        norm = float(rating)
        return norm

    
## dropping nulls, renaming column and applying function


rt_df.dropna(subset=['rating_y'],inplace=True)    
rt_df['critic_rating'] = rt_df['rating_y'].apply(norm_ratings)

Cleaning Runtimes and Release dates 

In [ ]:
# CLEAN RUNTIMES FLOAT>INT

def clean_runtime(runtime):
    minutes = runtime.split()[0]
    return int(minutes) 


rt_df.dropna(subset=['runtime'],inplace=True)    
rt_df['runtime'] = rt_df['runtime'].apply(clean_runtime)
rt_df.drop(columns = 'rating_y', inplace = True)
rt_df.head()

In [ ]:
# def clean_date(date):
#     temp = date[8:]
#     return float(temp)

# clean_date('Aug 17, 2012')

# # rt_df['year'] = rt_df['theater_date'].apply(clean_date)
# # rt_df

In [ ]:
# ADDING DIRECTOR NAME ID TO RT_DF

name_key = file_dict['imdb.name.basics'][['nconst', 'primary_name']]




rt_df2 = rt_df.merge(name_key, left_on = 'director', right_on = 'primary_name')
rt_df2 = rt_df2.drop(columns = 'primary_name')

rt_df2.head()

In [ ]:
movie_director_code = file_dict['imdb.title.crew'][['tconst', 'directors']]
movie_director_code['director'] = movie_director_code['directors'].str.split(',')
movie_director_code.drop(columns = 'directors', inplace = True)
movie_director_code = movie_director_code.explode('director')



In [ ]:
movie_director_code.head(1)

In [ ]:
file_dict['imdb.title.basics'].head(1)

In [ ]:
movie_id_names_time = file_dict['imdb.title.basics'][['tconst', 'primary_title','start_year', 'original_title', 'runtime_minutes']]
half_way_clean = movie_id_names_time.merge(movie_director_code, how = 'inner', on = 'tconst')
half_way_clean.head(1)

In [ ]:
half_way_clean.info()

In [ ]:
def float_conv(flt):
    integer = int(flt)
    return integer


half_way_clean.dropna(subset = ['runtime_minutes'], inplace = True)
half_way_clean['runtime_minutes'] = half_way_clean['runtime_minutes'].apply(float_conv)
half_way_clean.head(1)

In [ ]:
# len(rt_df2['id'].unique()) ## 663 unique
rt_df2.head(4)

In [ ]:
rt_df3 = rt_df2.dropna()


rt_df3.info()

In [410]:
# clean rt_df2 date to only include year 
# def clean_dates(date):
#     string = str(date)
#     norm = date[-4:]
#     return int(norm)

# type(clean_dates('Aug 17, 2012'))
# rt_df2['theater_date'].iloc([60])

def clean_dates(date):
    if type(date) == str:
        norm = date.split(',')[1]
        return int(norm)
    elif np.isnan(date) == True:
        return date
    elif type(date) == float:
        norm = int(date)
    
    else:
        errors = 'coerece'







# type(clean_dates('Aug 17, 2012'))
# clean_dates('Aug 17, 2012')





rt_df2 = rt_df3['theater_date'].apply(extract_year)
rt_df2.head()

0    2012
1    2012
2    2012
3    2012
4    2012
Name: theater_date, dtype: object

In [412]:
rt_df2

0        2012
1        2012
2        2012
3        2012
4        2012
         ... 
34758    1993
34759    1993
34760    1993
34761    1993
34762    1993
Name: theater_date, Length: 32737, dtype: object

In [336]:
# len(half_way_clean['tconst']) #133 k unique
half_way_clean.head(1)

,tconst,primary_title,start_year,original_title,runtime_minutes,director
0,tt0063540,Sunghursh,2013,Sunghursh,175,nm0712540


In [337]:
combined_df = rt_df2.merge(half_way_clean, left_on = ['runtime', 'nconst'], right_on = ['runtime_minutes', 'director'])

# len(combined_df['id'].unique())
combined_df.head()

,id,director_x,runtime,rating_x,theater_date,critic,critic_rating,nconst,tconst,primary_title,start_year,original_title,runtime_minutes,director_y
0,14,Taylor Hackford,117,R,"Jun 30, 2010",Laura Hiros,6.0,nm0000431,tt1125929,Love Ranch,2010,Love Ranch,117,nm0000431
1,14,Taylor Hackford,117,R,"Jun 30, 2010",James Luxford,4.0,nm0000431,tt1125929,Love Ranch,2010,Love Ranch,117,nm0000431
2,14,Taylor Hackford,117,R,"Jun 30, 2010",Brian Orndorf,5.0,nm0000431,tt1125929,Love Ranch,2010,Love Ranch,117,nm0000431
3,14,Taylor Hackford,117,R,"Jun 30, 2010",Nick Schager,5.0,nm0000431,tt1125929,Love Ranch,2010,Love Ranch,117,nm0000431
4,14,Taylor Hackford,117,R,"Jun 30, 2010",Frank Swietek,5.0,nm0000431,tt1125929,Love Ranch,2010,Love Ranch,117,nm0000431


In [340]:
# tester = combined_df.rename({'director_x': 'DirectorName', 'rating_x': "MPArating", 'nconst':'DirectorId'}, inplace = True)
# tester.head()
# combined_df


In [175]:
tester = combined_df.merge(file_dict['imdb.title.ratings'], on = 'tconst')
# tester2 = head()
# len(tester['id'].unique())
tester.info()
tester.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6686 entries, 0 to 6685
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               6686 non-null   int64  
 1   director_x       6686 non-null   object 
 2   runtime          6686 non-null   int64  
 3   rating_x         6686 non-null   object 
 4   critic           6494 non-null   object 
 5   critic_rating    6683 non-null   float64
 6   nconst           6686 non-null   object 
 7   tconst           6686 non-null   object 
 8   primary_title    6686 non-null   object 
 9   original_title   6686 non-null   object 
 10  runtime_minutes  6686 non-null   int64  
 11  director_y       6686 non-null   object 
 12  averagerating    6686 non-null   float64
 13  numvotes         6686 non-null   int64  
dtypes: float64(2), int64(4), object(8)
memory usage: 783.5+ KB


,id,director_x,runtime,rating_x,critic,critic_rating,nconst,tconst,primary_title,original_title,runtime_minutes,director_y,averagerating,numvotes
0,14,Taylor Hackford,117,R,Laura Hiros,6.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431,5.6,2742
1,14,Taylor Hackford,117,R,James Luxford,4.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431,5.6,2742
2,14,Taylor Hackford,117,R,Brian Orndorf,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431,5.6,2742
3,14,Taylor Hackford,117,R,Nick Schager,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431,5.6,2742
4,14,Taylor Hackford,117,R,Frank Swietek,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431,5.6,2742


In [179]:
# import os 
# os.chdir('')
!ls

Cleaning_Crtics.ipynb     file_list.txt             tmdb_imdb_gross.csv.gz
Full_data                 machine_learning          tmdb_movie_ids.csv
Untitled.ipynb            movie_ids_12_09_2021.json zippedData
Zip                      tmdb_imdb_gross.csv


In [177]:
# full_gross = pd.read_csv('data/tmdb_imdb_gross_full.csv')

In [321]:
income_data = pd.read_csv('Full_data') 
income_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8235 entries, 0 to 8234
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    8235 non-null   int64  
 1   id            8235 non-null   int64  
 2   tconst        7693 non-null   object 
 3   title         8235 non-null   object 
 4   release_date  8022 non-null   object 
 5   revenue       8235 non-null   float64
 6   budget        5488 non-null   float64
 7   net_profit    5488 non-null   float64
 8   year          8022 non-null   float64
 9   vote_average  8235 non-null   int64  
 10  popularity    8235 non-null   int64  
 11  vote_count    8235 non-null   int64  
 12  runtime       8180 non-null   float64
 13  genres        8235 non-null   object 
dtypes: float64(5), int64(5), object(4)
memory usage: 900.8+ KB


In [256]:
sum_data = tester.merge(income_data[['tconst', 'net_profit']], how = 'left', on = 'tconst')

In [272]:
sum_data = sum_data.dropna()

In [273]:
sum_data.tail(10)

,id,director_x,runtime,rating_x,critic,critic_rating,nconst,tconst,primary_title,original_title,runtime_minutes,director_y,averagerating,numvotes,net_profit
6676,1944,Michel Gondry,119,PG-13,Steve Persall,4.0,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273,5.8,148622,107817248.0
6677,1944,Michel Gondry,119,PG-13,Daniel Hubschman,4.0,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273,5.8,148622,107817248.0
6678,1944,Michel Gondry,119,PG-13,David Jenkins,6.0,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273,5.8,148622,107817248.0
6679,1944,Michel Gondry,119,PG-13,Peter Keough,5.0,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273,5.8,148622,107817248.0
6680,1944,Michel Gondry,119,PG-13,Keith Uhlich,4.0,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273,5.8,148622,107817248.0
6681,1944,Michel Gondry,119,PG-13,Edward Douglas,8.5,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273,5.8,148622,107817248.0
6682,1944,Michel Gondry,119,PG-13,Matt Singer,6.0,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273,5.8,148622,107817248.0
6683,1944,Michel Gondry,119,PG-13,Perri Nemiroff,6.0,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273,5.8,148622,107817248.0
6684,1944,Michel Gondry,119,PG-13,Sam Bathe,6.0,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273,5.8,148622,107817248.0
6685,1944,Michel Gondry,119,PG-13,Nell Minow,7.0,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273,5.8,148622,107817248.0


In [274]:
id_group = sum_data.groupby('primary_title')

In [275]:
critic_rating_by_movie = id_group['critic_rating'].mean()
pop_rating_by_movie = id_group['averagerating'].mean()
profit_by_movie = id_group['net_profit'].mean()

profit_list = profit_by_movie.tolist()
pop_rating_list = pop_rating_by_movie.tolist()
critic_rating_list = critic_rating_by_movie.tolist()
# critic_rating_by_movie

In [276]:
pop_rating_by_movie = id_group['averagerating'].mean()
# pop_rating_by_movie

In [277]:
profit_by_movie = id_group['net_profit'].mean()
# type(profit_by_movie)

In [280]:
profit_list = profit_by_movie.tolist()
pop_rating_list = pop_rating_by_movie.tolist()
critic_rating_list = critic_rating_by_movie.tolist()
# profit_list

In [320]:
# import seaborn as sns
# sns.scatterplot(x="critic_rating", y="net_profit", data=id_group);